# Membership Inference Competition (MICO) @ IEEE SatML 2023: CIFAR-10

Welcome to the MICO competition!

This notebook will walk you through the process of creating and packaging a submission to one of the challenges.

Let's start by downloading and extracting the archive for the CIFAR-10 challenge.

**NOTE**: Public anonymous access to the competition data is disabled. 
Upon registering for the competition, you will be shown a URL with an embedded bearer token that you must use instead of the URL below.

In [ ]:
import os
import urllib

from torchvision.datasets.utils import download_and_extract_archive

url = "https://membershipinference.blob.core.windows.net/mico/cifar10.zip" 
filename = "cifar10.zip"
md5 = "c615b172eb42aac01f3a0737540944b1"

# WARNING: this will download and extract a 2.1GiB file, if not already present. Please save the file and avoid re-downloading it.
try:
    download_and_extract_archive(url=url, download_root=os.curdir, extract_root=None, filename=filename, md5=md5, remove_finished=False)
except urllib.error.HTTPError as e:
    print(e)
    print("Have you replaced the URL above with the one you got after registering?")

## Contents

The archive was extracted under the `cifar10` folder containing 3 sub-folders, one for each of the scenarios in the challenge:

- `cifar10_lo`  : Models trained with DP-SGD and a small privacy budget ($\epsilon \approx 4$) 
- `cifar10_hi`  : Models trained with DP-SGD and a large privacy budget ($\epsilon \approx 10$) 
- `cifar10_inf` : Models trained without differential privacy guarantee ($\epsilon = \infty$)

Each of these folders contains 3 other folders:

- `train`: Models with metadata allowing to reconstruct their full training datasets. Use these to develop your attacks without having to train your own models.
- `dev`: Models with metadata allowing to reconstruct just the set of challenge examples. Membership predictions for these challenges will be used to evaluate submissions during the competition and update the live scoreboard in CodaLab. 
- `final`: Models with metadata allowing to reconstruct just the set of challenge examples. Membership predictions for these challenges will be used to evaluate submissions when the competition closes and to determine the final ranking.

Each model folder in `train`, `dev`, and `final` contains a `model.pt` file with the model weights (a serialized PyTorch `state_dict`). There are 100 models in `train`, and 50 models in each of `dev` and `final`.

Models in the `train` folder come with 3 PRNG seeds used to reconstruct the set of member and non-member challenge examples, and the rest of the examples in the training dataset of the model. Additionally (and redundantly), a `solution.csv` file reveals the membership information of the challenge examples.

Models in the `dev` and `final` folders contain just 1 PRNG seed used to reconstruct the set of challenge examples, without revealing which were included in the training dataset.

We provide utilities to reconstruct the different data splits from provided seeds and to load models as classes inheriting from `torch.nn.Module`. If you use TensorFlow, JAX, or any other framework, you can easily convert the models to the appropriate format (e.g. using ONXX).

Here's a summary of how the contents are structured:

- `cifar10_lo`
  - `train`
      - `model_0`
        - `model.pt`: Serialized model weights
        - `seed_challenge`: PRNG seed used to select a list of 100 challenge examples
        - `seed_training`: PRNG seed used to select the non-challenge examples in the training dataset
        - `seed_membership`: PRNG seed used to split the set of challenge examples into members and non-members (100 of each)
        - `solution.csv`: Membership information of the challenge examples (`1` for member, `0` for non-member)
      - ...
  - `dev`
      - `model_100`
        - `model.pt`
        - `seed_challenge`
      - ...
  - `final`
    - `model_150`
      - `model.pt`
      - `seed_challenge`
    - ...
- `cifar10_hi`
  - ...
- `cifar10_inf`
  - ...

## Task

Your task as a competitor is to produce, for each model in `dev` and `final`, a CSV file listing your confidence scores (values between 0 and 1) for the membership of the challenge examples. You must save these scores in a `prediction.csv` file and place it in the same folder as the corresponding model. A submission to the challenge is an an archive containing just these `prediction.csv` files.

**You must submit predictions for both `dev` and `final` when you submit to CodaLab.**

In the following, we will show you how to compute predictions from a basic membership inference attack and package them as a submission archive. 

In [ ]:
import numpy as np
import torch
import csv

from tqdm.notebook import tqdm
from mico_competition import ChallengeDataset, load_cifar10, load_model

In [ ]:
CHALLENGE = "cifar10"
LEN_TRAINING = 50000
LEN_CHALLENGE = 100

scenarios = os.listdir(CHALLENGE)
phases = ['dev', 'final', 'train']

dataset = load_cifar10(dataset_dir="/data")

criterion = torch.nn.CrossEntropyLoss(reduction='none')

for scenario in tqdm(scenarios, desc="scenario"):
    for phase in tqdm(phases, desc="phase"):
        root = os.path.join(CHALLENGE, scenario, phase)
        for model_folder in tqdm(sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])), desc="model"):
            path = os.path.join(root, model_folder)
            challenge_dataset = ChallengeDataset.from_path(path, dataset=dataset, len_training=LEN_TRAINING)
            challenge_points = challenge_dataset.get_challenges()

            # This is where you plug in your membership inference attack
            # As an example, here is a simple loss threshold attack

            # Loss Threshold Attack
            model = load_model('cifar10', path)
            challenge_dataloader = torch.utils.data.DataLoader(challenge_points, batch_size=2*LEN_CHALLENGE)
            features, labels = next(iter(challenge_dataloader))
            output = model(features)
            predictions = -criterion(output, labels).detach().numpy()
            # Normalize to unit interval
            min_prediction = np.min(predictions)
            max_prediction = np.max(predictions)
            predictions = (predictions - min_prediction) / (max_prediction - min_prediction)

            assert np.all((0 <= predictions) & (predictions <= 1))

            with open(os.path.join(path, "prediction.csv"), "w") as f:
                 csv.writer(f).writerow(predictions)

## Scoring

Let's see how the attack does on `train`, for which we have the ground truth. 
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack. 

In [ ]:
from mico_competition.scoring import tpr_at_fpr, score, generate_roc, generate_table
from sklearn.metrics import roc_curve, roc_auc_score

FPR_THRESHOLD = 0.1

all_scores = {}
phases = ['train']

for scenario in tqdm(scenarios, desc="scenario"): 
    all_scores[scenario] = {}    
    for phase in tqdm(phases, desc="phase"):
        predictions = []
        solutions  = []

        root = os.path.join(CHALLENGE, scenario, phase)
        for model_folder in tqdm(sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])), desc="model"):
            path = os.path.join(root, model_folder)
            predictions.append(np.loadtxt(os.path.join(path, "prediction.csv"), delimiter=","))
            solutions.append(np.loadtxt(os.path.join(path, "solution.csv"),   delimiter=","))

        predictions = np.concatenate(predictions)
        solutions = np.concatenate(solutions)
        
        scores = score(solutions, predictions)
        all_scores[scenario][phase] = scores

Let's plot the ROC curve for the attack and see how the attack performed on different metrics

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

for scenario in scenarios:
    fpr = all_scores[scenario]['train']['fpr']
    tpr = all_scores[scenario]['train']['tpr']
    fig = generate_roc(fpr, tpr)
    fig.suptitle(f"{scenario}", x=-0.1, y=0.5)
    fig.tight_layout(pad=1.0)

In [ ]:
import pandas as pd

for scenario in scenarios:
    print(scenario)
    scores = all_scores[scenario]['train']
    scores.pop('fpr', None)
    scores.pop('tpr', None)
    display(pd.DataFrame([scores]))

## Packaging the submission

Now we can store the predictions into a zip file, which you can submit to CodaLab.

In [ ]:
import zipfile

phases = ['dev', 'final']

with zipfile.ZipFile("predictions_cifar10.zip", 'w') as zipf:
    for scenario in tqdm(scenarios, desc="scenario"): 
        for phase in tqdm(phases, desc="phase"):
            root = os.path.join(CHALLENGE, scenario, phase)
            for model_folder in tqdm(sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])), desc="model"):
                path = os.path.join(root, model_folder)
                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    zipf.write(file)
                else:
                    raise FileNotFoundError(f"`prediction.csv` not found in {path}. You need to provide predictions for all challenges")